## new analsis script 26 september 2019

### import libraries

In [64]:
import pandas as pd
import re

In [112]:
#functions:
#excluding parenthesis
def drop_parenthesis(thinktank):
    parenthesis = re.findall('\(.*?\)',thinktank)
    for p in parenthesis:
        thinktank = thinktank.replace(p,'')
        thinktank = thinktank.strip()
    return thinktank, parenthesis

#count years
def count_years(year_list):
    return len(year_list)


#### import files:

In [75]:
file_path = '../mcgann_ggttir_aktuell.xlsx'

In [76]:
df_regions = pd.read_excel(file_path,sheet_name='regions')
df_special_category = pd.read_excel(file_path,sheet_name='special_category')
df_reserch_area = pd.read_excel(file_path,sheet_name='research_area')

In [77]:
#extract parenthesis from region df:
df_regions['thinktanks_clean'],  df_regions['parenthesis']= zip(*df_regions['clean_think_tank_name'].apply(drop_parenthesis))

#extract parenthesis from region df:
df_special_category['thinktanks_clean'],  df_special_category['parenthesis']= zip(*df_special_category['clean_think_tank_name'].apply(drop_parenthesis))

#extract parenthesis from region df:
df_reserch_area['thinktanks_clean'],  df_reserch_area['parenthesis']= zip(*df_reserch_area['clean_think_tank_name'].apply(drop_parenthesis))

In [84]:
df_regions.head(2)

,Rank,Think Tank,clean_think_tank_name,Category,Year,Country,thinktanks_clean,parenthesis
0,1,brookings institution (united states),brookings institution (united states),Top 30 Think Tanks United States,2008,United States,brookings institution,[(united states)]
1,2,council on foreign relations,council on foreign relations,Top 30 Think Tanks United States,2008,United States,council on foreign relations,[]


In [85]:
df_special_category.head(2)

,Rank,Think tank,clean_think_tank_name,Category,Year,country,thinktanks_clean,parenthesis
0,1.0,cato institute usa,cato institute (usa),Think Tanks with the Most Innovative Policy/Id...,2008.0,United States,cato institute,[(usa)]
1,2.0,brookings institute usa,brookings institute usa,Think Tanks with the Most Innovative Policy/Id...,2008.0,United States,brookings institute usa,[]


In [86]:
df_special_category.head(2)

,Rank,Think tank,clean_think_tank_name,Category,Year,country,thinktanks_clean,parenthesis
0,1.0,cato institute usa,cato institute (usa),Think Tanks with the Most Innovative Policy/Id...,2008.0,United States,cato institute,[(usa)]
1,2.0,brookings institute usa,brookings institute usa,Think Tanks with the Most Innovative Policy/Id...,2008.0,United States,brookings institute usa,[]


## 2. Data Analysis

## 2.1 Categories over time:

In [120]:
df_sc_categories = df_special_category.groupby(['Category'])['Year'].unique()
df_sc_categories = df_sc_categories.to_frame().reset_index()

In [121]:
df_sc_categories['nr_of_years'] = df_sc_categories['Year'].apply(count_years) 

In [122]:
df_sc_categories.sort_values(by='nr_of_years',ascending=False).head()

,Category,Year,nr_of_years
6,Best Government Affiliated Think Tanks,"[2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015....",9
71,Think Tanks with the Most Innovative Policy Id...,"[2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016....",8
73,Think Tanks with the Most Significant Impact o...,"[2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017....",7
11,Best Managed Think Tanks,"[2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0]",6
62,Think Tanks with the Best External Relations/P...,"[2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2018.0]",6


In [127]:
df_unique_categories = df_special_category.groupby(['Year'])['Category'].unique()
df_unique_categories = df_unique_categories.to_frame().reset_index()

In [128]:
df_unique_categories['nr_categories'] = df_unique_categories['Category'].

,Year,Category
0,2008.0,[Think Tanks with the Most Innovative Policy/I...
1,2009.0,[Think Tanks with the Most Innovative Policy/I...
2,2010.0,[Think Tanks with the Most Innovative Policy/I...
3,2011.0,[Think Tanks with the Most Innovative Policy I...
4,2012.0,[Think Tanks with the Most Innovative Policy I...
5,2013.0,"[Best Advocacy Campaign, Best For Profit Think..."
6,2014.0,"[Best Advocacy Campaign, Best For Profit Think..."
7,2015.0,"[Best Advocacy Campaign, Best For Profit Think..."
8,2016.0,"[Best Advocacy Campaign, Best For Profit Think..."
9,2017.0,"[Best Advocacy Campaign, Best For Profit Think..."
